## Pré-processamento da base de dados ANVISA

### Carregando a base de dados

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data_path = '../datasets/anvisa/'
data_file = 'TA_PRECOS_MEDICAMENTOS.csv'

cols = [4,5,6,8]
df = pd.read_csv('{}{}'.format(data_path, data_file), 
                 sep=';',
                 usecols=cols,
                 encoding='iso-8859-1')
df.shape

(53437, 4)

In [3]:
cols = ['ean', 'produto', 'apresentacao', 'principio_ativo']
df.columns = cols
df.head()

,ean,produto,apresentacao,principio_ativo
0,7896226102634,OSTENAN,10 MG COMP REV CX COM 30,ALENDRONATO DE SÓDIO TRI-HIDRATADO
1,7896226102627,OSTENAN,10 MG COMP REV CX COM 15,ALENDRONATO DE SÓDIO TRI-HIDRATADO
2,7730711011233,TEICOPLANINA,400 MG PÓ LIOFILIZADO SOL INJ CX C/ 1 FR/AMP X...,NC/NI
3,7896226100449,AMBROTEN,XPE ADU FR COM 100ML,ambroxol
4,7730711011493,OXALIPLATINA,50 MG PÓ LIOFILIZADO FR/AMP X 500 MG,NC/NI


### Removendo registros que não possuem o atributo EAN

Se não possui EAN ou EAN é inválido, DROP

In [4]:
# Removendo rows com EAN nulo
df.dropna(subset=['ean'], inplace=True)
df.shape

(52922, 4)

In [5]:
# Removendo rows em que o EAN não é numérico
df = df[df['ean'].str.isdigit()]
df.shape

(52643, 4)

In [6]:
# Removendo rows com EANs inválidos (conforme regra do cálculo do dígito verificador)
def is_ean_valid(row):
    return not sum(int(d)*i for d,i in zip(row['ean'],[1,3]*7))%10

indices = df.apply(is_ean_valid, axis=1)
df = df[indices]
df.shape

(51745, 4)

### Removendo registros que não possuem o atributo apresentação

In [7]:
df.dropna(subset=['apresentacao'], inplace=True)
df.shape

(51745, 4)

In [8]:
non_values = ['NC/NI', 'SI/NC']
df[df['apresentacao'].isin(non_values)]

,ean,produto,apresentacao,principio_ativo


### Removendo registros que não possuem os atributos produto e principio ativo (conectivo AND)

Se não possui produto e principio ativo, DROP (se tiver pelo menos um dos dois não dropa)

In [9]:
df.dropna(subset=['produto', 'principio_ativo'], how='all', inplace=True)
df.shape

(51745, 4)

In [10]:
df[df['produto'].isin(non_values) & df['principio_ativo'].isin(non_values)]

,ean,produto,apresentacao,principio_ativo


### UPPER

In [11]:
df['produto'] = df['produto'].str.upper()
df['apresentacao'] = df['apresentacao'].str.upper()
df['principio_ativo'] = df['principio_ativo'].str.upper()

### Removendo duplicatas

In [12]:
df[df.duplicated()].head()

,ean,produto,apresentacao,principio_ativo
7460,7896255716765,RITMONORM,"EV 3,5MG/ML SOL INJ CT 6 AMP VD AMB X 20ML",CLORIDRATO DE PROPAFENONA
7963,7896255701471,ATENOLOL,50 MG COM CT 2 BL AL PLAS INC X 14,ATENOLOL
7964,7896255701488,ATENOLOL,100 MG COM CT 2 BL AL PLAS INC X 14,ATENOLOL
9357,7896004700274,TENOXICAM,20 MG COM REV CT BL AL PLAS INC X 10,TENOXICAM
10210,7898919447695,GLICOSE MONOIDRATADA,100 MG/ML SOL INJ IV BOLS PP TRANS SIST FECH X...,GLICOSE MONOIDRATADA


In [13]:
df[df['apresentacao'] == 'EV 3,5MG/ML SOL INJ CT 6 AMP VD AMB X 20ML']

,ean,produto,apresentacao,principio_ativo
6838,7896255716765,RITMONORM,"EV 3,5MG/ML SOL INJ CT 6 AMP VD AMB X 20ML",CLORIDRATO DE PROPAFENONA
7460,7896255716765,RITMONORM,"EV 3,5MG/ML SOL INJ CT 6 AMP VD AMB X 20ML",CLORIDRATO DE PROPAFENONA


In [14]:
df.drop_duplicates(inplace=True)
df.shape

(50927, 4)

### Estatisticas

In [15]:
df.count()

ean                50927
produto            50927
apresentacao       50927
principio_ativo    50792
dtype: int64

In [16]:
df['ean'].unique().shape

(48728,)

### Gravando em arquivo

In [17]:
data_file = 'anvisa.csv'

pd.DataFrame(df).to_csv('{}{}'.format(data_path, data_file), 
                        sep=';',
                        index=False,
                        encoding='utf-8')